In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!pip install pytorch-ood

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, CIFAR100, SVHN, Places365

from pytorch_ood.dataset.img import (
    LSUNCrop,
    Textures,
    TinyImageNetCrop,
    GaussianNoise
)
from pytorch_ood.detector import (
    ODIN,
    EnergyBased,
    Entropy,
    KLMatching,
    Mahalanobis,
    MaxLogit,
    MaxSoftmax,
    ViM,
)
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import OODMetrics, ToRGB, ToUnknown

torch.manual_seed(1234)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([
    tvt.Resize(size=(32, 32)),
    ToRGB(),
    tvt.ToTensor(),
    tvt.Normalize(std=std, mean=mean)
])

In [4]:
def calculate(test_loader, detector):
    metrics = OODMetrics() # Evaluate Detectors

    print("Total count:", len(test_loader))
    cnt = 0
    for x, y in test_loader:
        metrics.update(detector(x.cuda()), y)
        cnt += 1
        if (cnt + 1) % 50 == 0:
            print(f"[{cnt}/{len(test_loader)}]")

    r = metrics.compute()
    print(r)
    return r

In [5]:
def test_with_all_datasets(detector, base_dataset, transform=trans, batch_size=128, num_workers=20):
    dataset_names = ["SVHN", "Textures", "LSUNCrop", "TinyImageNetCrop", "Places365", "GaussianNoise"]
    results = []

    for name in dataset_names:
        if name == "SVHN":
            dataset_out_test = SVHN(root="data", split="test", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Textures":
            dataset_out_test = Textures(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "LSUNCrop":
            dataset_out_test = LSUNCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "TinyImageNetCrop":
            dataset_out_test = TinyImageNetCrop(root="data", download=True, transform=transform, target_transform=ToUnknown())
        elif name == "Places365":
            dataset_out_test = Places365(root="data", split="val", small=True, transform=transform, target_transform=ToUnknown())
        elif name == "GaussianNoise":
            dataset_out_test = GaussianNoise(length=10000, transform=transform, target_transform=ToUnknown(), download=True)
        else:
            continue

        test_loader = DataLoader(torch.utils.data.ConcatDataset([base_dataset, dataset_out_test]), batch_size=batch_size, num_workers=num_workers)
        print()
        print(name)
        results.append(calculate(test_loader, detector))

    return results

### MaxSoftmax

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = MaxSoftmax(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Downloading: "https://github.com/wetliu/energy_ood/raw/master/CIFAR/snapshots/pretrained/cifar100_wrn_pretrained_epoch_99.pt" to /root/.cache/torch/hub/checkpoints/wrn-cifar100-pt.pt
100%|██████████| 8.66M/8.66M [00:00<00:00, 313MB/s]


100%|██████████| 169001437/169001437 [00:13<00:00, 12924006.83it/s]


Extracting data/cifar-100-python.tar.gz to data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


100%|██████████| 64275384/64275384 [00:40<00:00, 1592707.03it/s]


...

Extracting data/textures-r1_0_1.tar.gz to data

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7354907989501953, 'AUPR-IN': 0.5750514268875122, 'AUPR-OUT': 0.8314752578735352, 'FPR95TPR': 0.7139000296592712}


100%|██████████| 17309383/17309383 [00:00<00:00, 19451191.68it/s]


Extracting data/LSUN.tar.gz to data

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8558781147003174, 'AUPR-IN': 0.8435630202293396, 'AUPR-OUT': 0.874040961265564, 'FPR95TPR': 0.47130000591278076}


100%|██████████| 26501958/26501958 [00:01<00:00, 19744713.63it/s]


Extracting data/Imagenet.tar.gz to data

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8632166981697083, 'AUPR-IN': 0.8480734825134277, 'AUPR-OUT': 0.8823279738426208, 'FPR95TPR': 0.4334999918937683}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7391895055770874, 'AUPR-IN': 0.8944536447525024, 'AUPR-OUT': 0.5008723139762878, 'FPR95TPR': 0.6991000175476074}

GaussianNoise
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8067207336425781, 'AUPR-IN': 0.6909393072128296, 'AUPR-OUT': 0.8663459420204163, 'FPR95TPR': 0.39160001277923584}
AUROC: 0.7857130368550619
FPR: 0.5670500099658966


### ODIN

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = ODIN(model, eps=0.002)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.646986186504364, 'AUPR-IN': 0.7865357398986816, 'AUPR-OUT': 0.5167717933654785, 'FPR95TPR': 0.736299991607666}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7261209487915039, 'AUPR-IN': 0.5729402303695679, 'AUPR-OUT': 0.8262160420417786, 'FPR95TPR': 0.724399983882904}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8569043874740601, 'AUPR-IN': 0.8468577861785889, 'AUPR-OUT': 0.8752673268318176, 'FPR95TPR': 0.460999995470047}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8736330270767212, 'AUPR-IN': 0.862382709980011, 'AUPR-OUT': 0.8899842500686646, 'FPR95TPR': 0.42250001430511475}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7308443188667297, 'AUPR-IN': 0.8893899321556091, 'AUPR-OUT': 0.49168

### Mahalanobis

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = Mahalanobis(model.features)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.85689377784729, 'AUPR-IN': 0.9309223890304565, 'AUPR-OUT': 0.7334612607955933, 'FPR95TPR': 0.5418999791145325}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.8992289304733276, 'AUPR-IN': 0.8528116345405579, 'AUPR-OUT': 0.9365952014923096, 'FPR95TPR': 0.4544000029563904}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.5203654170036316, 'AUPR-IN': 0.4674402177333832, 'AUPR-OUT': 0.5986384749412537, 'FPR95TPR': 0.8277999758720398}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.5595657825469971, 'AUPR-IN': 0.4922320544719696, 'AUPR-OUT': 0.6488621830940247, 'FPR95TPR': 0.7591999769210815}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.6392639875411987, 'AUPR-IN': 0.8424248099327087, 'AUPR-OUT': 0.351

### Energy

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = EnergyBased(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.7387329339981079, 'AUPR-IN': 0.8501717448234558, 'AUPR-OUT': 0.5959532260894775, 'FPR95TPR': 0.6674000024795532}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7629623413085938, 'AUPR-IN': 0.6157165765762329, 'AUPR-OUT': 0.8509080410003662, 'FPR95TPR': 0.7124999761581421}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9588834643363953, 'AUPR-IN': 0.9571850895881653, 'AUPR-OUT': 0.9623993635177612, 'FPR95TPR': 0.20309999585151672}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.952881932258606, 'AUPR-IN': 0.9492186307907104, 'AUPR-OUT': 0.9575362205505371, 'FPR95TPR': 0.21170000731945038}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7581775784492493, 'AUPR-IN': 0.9042642116546631, 'AUPR-OUT': 0.

### Entropy

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = Entropy(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.7309297919273376, 'AUPR-IN': 0.8555963039398193, 'AUPR-OUT': 0.5878225564956665, 'FPR95TPR': 0.6890000104904175}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7506520748138428, 'AUPR-IN': 0.6022067070007324, 'AUPR-OUT': 0.8380107283592224, 'FPR95TPR': 0.7127000093460083}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.892271101474762, 'AUPR-IN': 0.8919202089309692, 'AUPR-OUT': 0.8985163569450378, 'FPR95TPR': 0.4519999921321869}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8988495469093323, 'AUPR-IN': 0.8950001001358032, 'AUPR-OUT': 0.9065179824829102, 'FPR95TPR': 0.4108000099658966}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7520604133605957, 'AUPR-IN': 0.9018186926841736, 'AUPR-OUT': 0.50

### MaxLogit

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = MaxLogit(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.7395377159118652, 'AUPR-IN': 0.8536348342895508, 'AUPR-OUT': 0.5944435000419617, 'FPR95TPR': 0.6712999939918518}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7636523842811584, 'AUPR-IN': 0.6165502667427063, 'AUPR-OUT': 0.8512518405914307, 'FPR95TPR': 0.7138000130653381}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.9514684677124023, 'AUPR-IN': 0.9468564987182617, 'AUPR-OUT': 0.9569815993309021, 'FPR95TPR': 0.21719999611377716}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.946761429309845, 'AUPR-IN': 0.940183162689209, 'AUPR-OUT': 0.9531491994857788, 'FPR95TPR': 0.22439999878406525}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.759489893913269, 'AUPR-IN': 0.9050794839859009, 'AUPR-OUT': 0.52

### KLMatching

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = KLMatching(model)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified
Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282
[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.7014661431312561, 'AUPR-IN': 0.8676372170448303, 'AUPR-OUT': 0.4807525873184204, 'FPR95TPR': 0.7628999948501587}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.7185306549072266, 'AUPR-IN': 0.6307766437530518, 'AUPR-OUT': 0.7842214107513428, 'FPR95TPR': 0.8511999845504761}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.7772537469863892, 'AUPR-IN': 0.8041590452194214, 'AUPR-OUT': 0.6868447065353394, 'FPR95TPR': 0.945900022983551}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.8209725022315979, 'AUPR-IN': 0.8180630207061768, 'AUPR-OUT': 0.7937166094779968, 'FPR95TPR': 0.7091000080108643}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.6605174541473389, 'AUPR-IN': 0.8806449770927429, 'AUPR-OUT': 0.29

### ViM

In [ ]:
model = WideResNet(num_classes=100, pretrained="cifar100-pt").cuda().eval()
dataset_in_test = CIFAR100(root="data", train=False, download=True, transform=trans)
loader_in_train = DataLoader(dataset_in_test, batch_size=128, num_workers=20)

detector = ViM(model.features, d=64, w=model.fc.weight, b=model.fc.bias)
detector.fit(loader_in_train, device="cuda")
results = test_with_all_datasets(detector, dataset_in_test)

mean_AUROC = np.mean([r["AUROC"] for r in results])
mean_FPR = np.mean([r["FPR95TPR"] for r in results])

print("AUROC:", mean_AUROC)
print("FPR:", mean_FPR)

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Using downloaded and verified file: data/test_32x32.mat

SVHN
Total count: 282


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[49/282]
[99/282]
[149/282]
[199/282]
[249/282]
{'AUROC': 0.9254005551338196, 'AUPR-IN': 0.9664667844772339, 'AUPR-OUT': 0.8588016033172607, 'FPR95TPR': 0.32659998536109924}

Textures
Total count: 123
[49/123]
[99/123]
{'AUROC': 0.9068517088890076, 'AUPR-IN': 0.8625251054763794, 'AUPR-OUT': 0.9417524337768555, 'FPR95TPR': 0.41929998993873596}

LSUNCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.7555776238441467, 'AUPR-IN': 0.6573880910873413, 'AUPR-OUT': 0.811265766620636, 'FPR95TPR': 0.5440000295639038}

TinyImageNetCrop
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC': 0.7996347546577454, 'AUPR-IN': 0.715462863445282, 'AUPR-OUT': 0.8432332873344421, 'FPR95TPR': 0.48190000653266907}

Places365
Total count: 364
[49/364]
[99/364]
[149/364]
[199/364]
[249/364]
[299/364]
[349/364]
{'AUROC': 0.7013358473777771, 'AUPR-IN': 0.8769044280052185, 'AUPR-OUT': 0.41298708319664, 'FPR95TPR': 0.7990999817848206}

GaussianNoise
Total count: 157
[49/157]
[99/157]
[149/157]
{'AUROC':

### Outlier Exposure (OE)

In [16]:
from torch.optim import Adam
from pytorch_ood.dataset.img import Textures, TinyImages300k
from pytorch_ood.detector import MaxSoftmax
from pytorch_ood.loss import OutlierExposureLoss
from pytorch_ood.utils import OODMetrics, ToUnknown

torch.manual_seed(123)

# maximum number of epochs and training iterations
n_epochs = 10
device = "cuda:0"

# %%
# Setup preprocessing and data
trans = tvt.Compose([tvt.Resize(size=(32, 32)), tvt.ToTensor()])

# setup IN training data
dataset_in_train = CIFAR100(root="data", train=True, download=True, transform=trans)

# setup OOD training data, use ToUnknown() to mark labels as OOD
# this way, outlier exposure can automatically decide if the training samples are IN or OOD
dataset_out_train = TinyImages300k(
    root="data", download=True, transform=trans, target_transform=ToUnknown()
)

# setup IN test data
dataset_in_test = CIFAR100(root="data", train=False, transform=trans)

# setup OOD test data, use ToUnknown() to mark labels as OOD
dataset_out_test = Textures(
    root="data", download=True, transform=trans, target_transform=ToUnknown()
)

# create data loaders
train_loader = DataLoader(
    dataset_in_train + dataset_out_train, batch_size=64, shuffle=True
)
test_loader = DataLoader(dataset_in_test + dataset_out_test, batch_size=64)

# %%
# Create DNN, pretrained on the imagenet excluding cifar100 classes
model = WideResNet(num_classes=1000, pretrained="imagenet32-nocifar")
# we have to replace the final layer to account for the lower number of
# classes in the CIFAR100 dataset
model.fc = torch.nn.Linear(model.fc.in_features, 100)

model.to(device)

opti = Adam(model.parameters())
criterion = OutlierExposureLoss(alpha=0.5)


# %%
# Define a function to test the model
def test():
    softmax = MaxSoftmax(model)

    metrics_softmax = OODMetrics()
    model.eval()

    with torch.no_grad():
        for x, y in test_loader:
            metrics_softmax.update(softmax(x.to(device)), y)

    print(metrics_softmax.compute())
    model.train()


# %%
# Start training
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    for x, y in train_loader:
        logits = model(x.to(device))
        loss = criterion(logits, y.to(device))
        opti.zero_grad()
        loss.backward()
        opti.step()

    test()

Files already downloaded and verified
Epoch 0
{'AUROC': 0.6468272805213928, 'AUPR-IN': 0.5213726758956909, 'AUPR-OUT': 0.7458751201629639, 'FPR95TPR': 0.8876000046730042}
Epoch 1
{'AUROC': 0.8347277641296387, 'AUPR-IN': 0.7417207956314087, 'AUPR-OUT': 0.8896232843399048, 'FPR95TPR': 0.6370999813079834}
Epoch 2
{'AUROC': 0.7767816781997681, 'AUPR-IN': 0.6278778314590454, 'AUPR-OUT': 0.850523829460144, 'FPR95TPR': 0.6934000253677368}
Epoch 3
{'AUROC': 0.823417067527771, 'AUPR-IN': 0.7490994334220886, 'AUPR-OUT': 0.8821779489517212, 'FPR95TPR': 0.6647999882698059}
Epoch 4
{'AUROC': 0.8599973320960999, 'AUPR-IN': 0.7726024389266968, 'AUPR-OUT': 0.9102161526679993, 'FPR95TPR': 0.5281000137329102}
Epoch 5
{'AUROC': 0.8633450269699097, 'AUPR-IN': 0.7908233404159546, 'AUPR-OUT': 0.9105650186538696, 'FPR95TPR': 0.5515999794006348}
Epoch 6
{'AUROC': 0.8534408807754517, 'AUPR-IN': 0.7555985450744629, 'AUPR-OUT': 0.9091955423355103, 'FPR95TPR': 0.5296000242233276}
Epoch 7
{'AUROC': 0.8800964951515

In [17]:
def test(test_loader, name):
    softmax = MaxSoftmax(model)

    metrics_softmax = OODMetrics()
    model.eval()

    with torch.no_grad():
        for x, y in test_loader:
            metrics_softmax.update(softmax(x.to(device)), y)
    print(name)
    print(metrics_softmax.compute())

In [18]:
dataset_names = ["SVHN", "Textures", "LSUNCrop", "TinyImageNetCrop", "Places365", "GaussianNoise"]
datasets_out_test = {
    "SVHN": SVHN(root="./data", split="test", download=True, transform=trans, target_transform=ToUnknown()),
    "Textures": Textures(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "LSUNCrop": LSUNCrop(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "TinyImageNetCrop": TinyImageNetCrop(root="./data", download=True, transform=trans, target_transform=ToUnknown()),
    "Places365": Places365(root="./data", split="val", small=True, download=False, transform=trans, target_transform=ToUnknown()),
    "GaussianNoise": GaussianNoise(length=10000, transform=trans, target_transform=ToUnknown())
}

for name, dataset in datasets_out_test.items():
    dataset_in_test =  CIFAR100(root="data", train=False, transform=trans)
    test_loader = DataLoader(dataset_in_test + dataset, batch_size=64)
    test(test_loader, name)

Using downloaded and verified file: ./data/test_32x32.mat
SVHN
{'AUROC': 0.9353353977203369, 'AUPR-IN': 0.9653217792510986, 'AUPR-OUT': 0.8990997076034546, 'FPR95TPR': 0.22689999639987946}
Textures
{'AUROC': 0.8684343099594116, 'AUPR-IN': 0.7733821272850037, 'AUPR-OUT': 0.915361762046814, 'FPR95TPR': 0.47769999504089355}
LSUNCrop
{'AUROC': 0.9128215909004211, 'AUPR-IN': 0.896831750869751, 'AUPR-OUT': 0.9203508496284485, 'FPR95TPR': 0.3393000066280365}
TinyImageNetCrop
{'AUROC': 0.9196836948394775, 'AUPR-IN': 0.9047382473945618, 'AUPR-OUT': 0.9285298585891724, 'FPR95TPR': 0.3073999881744385}
Places365
{'AUROC': 0.8459770679473877, 'AUPR-IN': 0.9435319900512695, 'AUPR-OUT': 0.6646593809127808, 'FPR95TPR': 0.5497999787330627}
GaussianNoise
{'AUROC': 0.9485790133476257, 'AUPR-IN': 0.8635244369506836, 'AUPR-OUT': 0.9705520272254944, 'FPR95TPR': 0.07900000363588333}
